In [1]:
%matplotlib inline
import os
import collections
import torch
import torchvision
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt

from NYUDv2Loader import *

os.environ['CUDA_VISIBLE_DEVICES'] = '3'
data_path = '/home/dongwonshin/Desktop/Datasets/NYUDv2/'
arg_string = '--arch mynetwork20180516 --batch_size 3'

# Argument setting

In [2]:
import sys, os
import torch
import visdom
import argparse
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torch.autograd import Variable
from torch.utils import data
from tqdm import tqdm

from ptsemseg.models import get_model
from ptsemseg.loader import get_loader, get_data_path
from ptsemseg.metrics import runningScore
from ptsemseg.loss import *
from ptsemseg.augmentations import *


parser = argparse.ArgumentParser(description='Hyperparams')
parser.add_argument('--arch', nargs='?', type=str, default='fcn8s', help='Architecture to use [\'fcn8s, unet, segnet etc\']')
parser.add_argument('--img_rows', nargs='?', type=int, default=256, help='Height of the input image')
parser.add_argument('--img_cols', nargs='?', type=int, default=256, help='Width of the input image')

parser.add_argument('--img_norm', dest='img_norm', action='store_true', help='Enable input image scales normalization [0, 1] | True by default')
parser.add_argument('--no-img_norm', dest='img_norm', action='store_false', help='Disable input image scales normalization [0, 1] | True by default')
parser.set_defaults(img_norm=True)

parser.add_argument('--n_epoch', nargs='?', type=int, default=100, help='# of the epochs')
parser.add_argument('--batch_size', nargs='?', type=int, default=1, help='Batch Size')
parser.add_argument('--l_rate', nargs='?', type=float, default=1e-5, help='Learning Rate')
parser.add_argument('--feature_scale', nargs='?', type=int, default=1, help='Divider for # of features to use')
parser.add_argument('--resume', nargs='?', type=str, default=None, help='Path to previous saved model to restart from')

parser.add_argument('--visdom', dest='visdom', action='store_true', help='Enable visualization(s) on visdom | False by default')
parser.add_argument('--no-visdom', dest='visdom', action='store_false', help='Disable visualization(s) on visdom | False by default')
parser.set_defaults(visdom=False)

In [3]:
args = parser.parse_args(arg_string.split(' '))

# Setup Augmentations
data_aug= Compose([RandomRotate(10), RandomHorizontallyFlip()])

# Setup Dataloader
t_loader = NYUDv2Loader(data_path, is_transform=True)
v_loader = NYUDv2Loader(data_path, is_transform=True, split='val')

n_classes = t_loader.n_classes
trainloader = data.DataLoader(t_loader, batch_size=args.batch_size, num_workers=16, shuffle=True)
valloader = data.DataLoader(v_loader, batch_size=args.batch_size, num_workers=16)

# Setup Metrics
running_metrics = runningScore(n_classes)

# Setup visdom for visualization
if args.visdom:
    vis = visdom.Visdom()

    loss_window = vis.line(X=torch.zeros((1,)).cpu(),
                       Y=torch.zeros((1)).cpu(),
                       opts=dict(xlabel='minibatches',
                                 ylabel='Loss',
                                 title='Training Loss',
                                 legend=['Loss']))

# Setup Model
model = get_model(args.arch, n_classes)

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.cuda()

# Check if model has custom optimizer / loss
if hasattr(model.module, 'optimizer'):
    optimizer = model.module.optimizer
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=args.l_rate, momentum=0.99, weight_decay=5e-4)

if hasattr(model.module, 'loss'):
    print('Using custom loss')
    loss_fn = model.module.loss
else:
    loss_fn = cross_entropy2d

if args.resume is not None:                                         
    if os.path.isfile(args.resume):
        print("Loading model and optimizer from checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        print("Loaded checkpoint '{}' (epoch {})"                    
              .format(args.resume, checkpoint['epoch']))
    else:
        print("No checkpoint found at '{}'".format(args.resume)) 


# Training

In [4]:
best_iou = -100.0 
for epoch in range(args.n_epoch):
    model.train()
    for i, (color_imgs, depth_imgs, label_imgs) in enumerate(trainloader):
        images = Variable(color_imgs.cuda())
        depth_images = Variable(depth_imgs.cuda())
        labels = Variable(label_imgs.cuda())

        optimizer.zero_grad()
        outputs = model(images,depth_images)

        loss = loss_fn(input=outputs, target=labels)

        loss.backward()
        optimizer.step()

        if args.visdom:
            vis.line(
                X=torch.ones((1, 1)).cpu() * i,
                Y=torch.Tensor([loss.data[0]]).unsqueeze(0).cpu(),
                win=loss_window,
                update='append')

        if (i+1) % 20 == 0:
            print("Epoch [%d/%d] Loss: %.4f" % (epoch+1, args.n_epoch, loss.data[0]))

    model.eval()
    for i_val, (color_images_val, depth_images_val, label_images_val) in tqdm(enumerate(valloader)):
        color_images_val = Variable(color_images_val.cuda(), volatile=True)
        depth_images_val = Variable(depth_images_val.cuda(), volatile=True)
        label_images_val = Variable(label_images_val.cuda(), volatile=True)

        outputs = model(color_images_val, depth_images_val)
        pred = outputs.data.max(1)[1].cpu().numpy()
        gt = label_images_val.data.cpu().numpy()
        running_metrics.update(gt, pred)

    score, class_iou = running_metrics.get_scores()
    for k, v in score.items():
        print(k, v)
    running_metrics.reset()

    if score['Mean IoU : \t'] >= best_iou:
        best_iou = score['Mean IoU : \t']
        state = {'epoch': epoch+1,
                 'model_state': model.state_dict(),
                 'optimizer_state' : optimizer.state_dict(),}
        torch.save(state, "../model_weights/{}_{}_best_model.pkl".format(args.arch, 'NYUDv2'))

/home/dongwonshin/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1462: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.")


Epoch [1/100] Loss: 2.9554
Epoch [1/100] Loss: 2.5272
Epoch [1/100] Loss: 2.4025
Epoch [1/100] Loss: 2.0798
Epoch [1/100] Loss: 2.0811
Epoch [1/100] Loss: 2.2976
Epoch [1/100] Loss: 2.2072
Epoch [1/100] Loss: 2.0583
Epoch [1/100] Loss: 2.3417
Epoch [1/100] Loss: 1.6881
Epoch [1/100] Loss: 1.5705
Epoch [1/100] Loss: 1.7858
Epoch [1/100] Loss: 1.6432


218it [01:07,  3.23it/s]


Overall Acc: 	 0.375063735386
Mean Acc : 	 0.164711411726
FreqW Acc : 	 0.199694344237
Mean IoU : 	 0.0979635496264
Epoch [2/100] Loss: 1.6958
Epoch [2/100] Loss: 1.4962
Epoch [2/100] Loss: 1.5690
Epoch [2/100] Loss: 1.8898
Epoch [2/100] Loss: 1.8617
Epoch [2/100] Loss: 1.8130
Epoch [2/100] Loss: 1.6321
Epoch [2/100] Loss: 1.5643
Epoch [2/100] Loss: 1.4530
Epoch [2/100] Loss: 1.4526
Epoch [2/100] Loss: 1.3476
Epoch [2/100] Loss: 1.5633
Epoch [2/100] Loss: 2.0625


218it [01:08,  3.20it/s]


...

Overall Acc: 	 0.55391918124
Mean Acc : 	 0.45311533661
FreqW Acc : 	 0.401335490525
Mean IoU : 	 0.337973632961


Epoch [61/100] Loss: 0.3783
Epoch [61/100] Loss: 0.3871
Epoch [61/100] Loss: 0.4220
Epoch [61/100] Loss: 0.4229
Epoch [61/100] Loss: 0.4829
Epoch [61/100] Loss: 0.2973
Epoch [61/100] Loss: 0.4879
Epoch [61/100] Loss: 0.5099
Epoch [61/100] Loss: 0.4269
Epoch [61/100] Loss: 0.4472
Epoch [61/100] Loss: 0.4088
Epoch [61/100] Loss: 0.3234
Epoch [61/100] Loss: 0.4449


218it [01:09,  3.13it/s]


Overall Acc: 	 0.566945151233
Mean Acc : 	 0.481044248853
FreqW Acc : 	 0.409128080021
Mean IoU : 	 0.349615004812
Epoch [62/100] Loss: 0.3861
Epoch [62/100] Loss: 0.4082
Epoch [62/100] Loss: 0.4167


FileNotFoundError: Traceback (most recent call last):
  File "/home/dongwonshin/.conda/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/dongwonshin/.conda/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/dongwonshin/Desktop/semantic_segmentation_rgbd/research_notebooks/NYUDv2Loader.py", line 59, in __getitem__
    depth_img = np.load(depth_path)
  File "/home/dongwonshin/.conda/envs/pytorch/lib/python3.6/site-packages/numpy/lib/npyio.py", line 370, in load
    fid = open(file, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/home/dongwonshin/Desktop/Datasets/NYUDv2/train/depth/nyu_depth_0369.npy'


# Training.. fcn8s_rgbd_maskedconv on NYUDv2

학습시키다가.. 도중에 npy를 png로 바꿔서.. 중단됨 
근데 어차피.. 잘 학습 되지 않았던거 같아서..
멈춰도됨..
뎁스영상 png로 했으니.. 다시 학습시켜볼까..

HHA 영상도 만들어야하는데..